In [1]:
import os
print(os.getcwd())
os.chdir("../")
print(os.getcwd())

/home/xchen/Projects/XMutant/XMutant-IMDB/data_analysis
/home/xchen/Projects/XMutant/XMutant-IMDB


In [30]:
import pandas as pd

csv_name_r = "random.csv"
csv_name_ig = "integratedgradients.csv"
csv_name_l = "lime.csv"
csv_name_sg = "smoothgrad.csv"

df_r = pd.read_csv(os.path.join("csv_results", csv_name_r))
df_ig = pd.read_csv(os.path.join("csv_results", csv_name_ig))
df_lime = pd.read_csv(os.path.join("csv_results", csv_name_l))
df_sg = pd.read_csv(os.path.join("csv_results", csv_name_sg))

In [3]:
df_r

id  generation     elapsed  misbehaviour  expected_label  \
0     0          39   76.137438          True               1   
1     1          94  165.655379          True               0   
2     5          20   43.236679          True               1   
3    14           8   21.108134          True               0   
4    15          40   77.825763          True               0   
5    22          14   32.437293          True               1   
6    25           2    9.442095          True               1   
7    30           3   11.443250          True               1   
8    32          12   28.733663          True               0   
9    33          44   84.717555          True               0   
10   38          52   98.091172          True               1   
11   42           3   11.426969          True               1   
12   50          60  111.319568          True               0   
13   52           7   19.140404          True               0   
14   56          62  114.501763          True               0   
15   62          57  106.320460          True               0   
16   66          34   67.431721          True               1   
17   80          17   37.766198          True               1   
18   83          89  157.724341          True               0   
19   86          59  109.597333          True               1   
20   87          13   30.504286          True               0   
21   89          37   72.547462          True               0   
22  102          14   32.290278          True               0   
23  113          18   39.483043          True               1   
24  115          87  154.514910          True               0   
25  118          33   65.607523          True               0   
26  120           3   11.283304          True               0   
27  121           2    9.264281          True               0   
28  124          63  115.978858          True               0   
29  125          14   32.252434          True               1   
30  127          49   92.847062          True               0   
31  132          10   24.733988          True               0   
32  135          25   51.777190          True               0   
33  137          32   63.860597          True               0   
34  146          21   44.734956          True               0   
35  147          39   75.861304          True               1   
36  149           2    9.218377          True               0   
37  150          26   53.509118          True               1   
38  152          76  136.876720          True               0   
39  155           4   13.171487          True               0   
40  156          41   79.542821          True               0   
41  158           9   22.766956          True               0   
42  163          41   79.536836          True               1   

    predicted_label                                               text  \
0                 0  <start> if you expect that this movie is full ...   
1                 1  <start> i have to say is the only reason i eve...   
2                 0  <start> this without any doubt is the greatest...   
3                 1  <start> one of the worst films of it's genre b...   
4                 1  <start> laughably awful one might think that a...   
5                 0  <start> being <unk> and brought up with the hi...   
6                 0  <start> such a shame that this wonderful brigh...   
7                 0  <start> i <unk> to the library to check out th...   
8                 1  <start> artificial melodrama with a screenplay...   
9                 1  <start> though it's better than most made for ...   
10                0  <start> this movie has remained in my mind for...   
11                0  <start> i'm torn about this show while most pa...   
12                1  <start> when i first saw race against fear don...   
13                1  <start> br br as a fan of bad movies and mst3k...   
14                1  <start> over 

In [33]:
NGEN = 100
import numpy as np

def cumulative_csv(df):
    col_names = ['generation', 'num', 'validity', 'preservation']

    df_cum = pd.DataFrame(columns= col_names)
    df_cum.generation = np.linspace(1,NGEN,NGEN)
    # df['generation'].cumsum()

    for i in range(len(df_cum)):
        df_cum.at[i, "num"] = (df['generation'] == i).sum()
        df_cum.at[i, "validity"] = ((df['generation'] == i) & (df['validity']>=0)).sum()
        df_cum.at[i, "preservation"] = ((df['generation'] == i) & (df['validity'] == df['expected_label'].astype('int'))).sum()

    df_cum['cum_num']  = df_cum['num'].cumsum()
    df_cum['cum_validity']  = df_cum['validity'].cumsum()
    df_cum['cum_preservation']  = df_cum['preservation'].cumsum()
    df_cum['auc_cum_num'] = df_cum['cum_num'].cumsum()
    return df_cum

In [34]:
df_cum_random = cumulative_csv(df_r)
df_cum_sg = cumulative_csv(df_sg)
df_cum_ig = cumulative_csv(df_ig)
df_cum_lime = cumulative_csv(df_lime)

In [35]:
df_cum_random

generation num validity preservation cum_num cum_validity  \
0          1.0   0        0            0       0            0   
1          2.0   0        0            0       0            0   
2          3.0   3        3            1       3            3   
3          4.0   3        3            2       6            6   
4          5.0   1        1            1       7            7   
..         ...  ..      ...          ...     ...          ...   
95        96.0   0        0            0      43           42   
96        97.0   0        0            0      43           42   
97        98.0   0        0            0      43           42   
98        99.0   0        0            0      43           42   
99       100.0   0        0            0      43           42   

   cum_preservation auc_cum_num  
0                 0           0  
1                 0           0  
2                 1           3  
3                 3           9  
4                 4          16  
..              ...         ...  
95               33        2753  
96               33        2796  
97               33        2839  
98               33        2882  
99               33        2925  

[100 rows x 8 columns]

In [71]:
methods = ["smoothgrad", "integratedgradients", "lime", "random"]
df_summary = pd.DataFrame(columns=["generation"] + methods)
df_summary['generation'] = np.linspace(9,99,10)
method_dfs =  {"smoothgrad":df_cum_sg, "integratedgradients":df_cum_ig, "lime":df_cum_lime, "random":df_cum_random}

In [72]:
np.linspace(9,99,10)

array([ 9., 19., 29., 39., 49., 59., 69., 79., 89., 99.])

In [73]:
df_tmp_random = pd.merge(df_cum_random[['generation', 'cum_num', 'auc_cum_num']],
                          df_summary['generation'], on='generation', how='right')

In [81]:
for met in methods:
    df_tmp = pd.merge(method_dfs[met][['generation', 'cum_num', 'auc_cum_num']], 
                      df_summary['generation'], on='generation', how='right')
    for i in range(len(df_summary)):
        df_summary.at[i, met] = ("& " +"{:.2f}".format(df_tmp.at[i, 'cum_num']/174*100) +'/' + 
        "{:.2f}".format(df_tmp.at[i, 'auc_cum_num']/df_tmp_random.at[i, 'auc_cum_num']) )

In [59]:
(df_tmp_random['cum_num']/174*100)


0     5.172414
1    10.344828
2    12.643678
3    14.942529
4    18.390805
5    20.114943
6    22.413793
7    22.988506
8    23.563218
9    24.712644
Name: cum_num, dtype: object

In [77]:
print("{:.2f}".format(df_tmp.at[i, 'cum_num']/174*100))
print("{:.2f}".format(df_tmp.at[i, 'auc_cum_num']/df_tmp_random.at[i, 'auc_cum_num']*100))

24.71
100.00


In [66]:
"{:.2f}".format(df_tmp_random.at[7, 'cum_num']/174*100)

'22.99'

In [82]:
df_summary

generation    smoothgrad integratedgradients          lime        random
0         9.0  & 14.37/2.21        & 17.24/2.70   & 6.90/1.32   & 5.17/1.00
1        19.0  & 22.99/2.41        & 37.93/3.43  & 13.79/1.29  & 10.34/1.00
2        29.0  & 31.61/2.41        & 48.28/3.56  & 18.39/1.32  & 12.64/1.00
3        39.0  & 35.06/2.43        & 55.17/3.63  & 20.11/1.35  & 14.94/1.00
4        49.0  & 37.93/2.31        & 60.34/3.52  & 22.41/1.31  & 18.39/1.00
5        59.0  & 39.66/2.23        & 60.92/3.41  & 24.71/1.28  & 20.11/1.00
6        69.0  & 41.95/2.15        & 63.79/3.29  & 25.86/1.25  & 22.41/1.00
7        79.0  & 44.83/2.11        & 65.52/3.21  & 27.01/1.23  & 22.99/1.00
8        89.0  & 45.98/2.08        & 67.24/3.16  & 29.89/1.24  & 23.56/1.00
9        99.0  & 48.28/2.06        & 68.39/3.11  & 29.89/1.23  & 24.71/1.00